# Feature Engineering
This notebook is used in order to generate the csv files, which data will be used in order to train and test the neural networks.

In [64]:
from os import listdir
import tensorflow as tf
import numpy as np
import pandas as pd
import datetime
import math

## Selecting Parameters

In [65]:
# ----------- Select yearas for training or testing and the name of the dataset (train or test)
start_year = 2016 # start year (2007 for train and 2016 for test)
end_year = 2017 # end year (2015 for train and 2017 for test)
train_or_test = 'test' # you must put here if the dataset corresponds to train or test 

## Creating Function that makes the feature engineering of the data

## Helper Functions

In [66]:
def excel_date(date1):
    temp = datetime.datetime(1899, 12, 30)    # Note, not 31st Dec but 30th!
    delta = date1 - temp
    return float(delta.days) + (float(delta.seconds) / 86400)


def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = listdir(path_to_dir)
    return [ filename for filename in filenames if filename.endswith( suffix ) ]

def coord_solar(numday, TL, longitude, latitude, DST):
    C1 = -5 #Time zone Colombia
    C2 = DST #Time change (0 for Colombia)
    
    # Declination
    
    B = (numday-1)*(360/365)
    dec = (180/np.pi)*(0.006918-0.399912*np.cos(B*(np.pi/180))+0.070257*np.sin(B*(np.pi/180))- \
    0.006758*np.cos(2*B*(np.pi/180))+0.000907*np.sin(2*B*(np.pi/180))- \
    0.02697*np.cos(3*B*(np.pi/180))+0.00148*np.sin(3*B*(np.pi/180)));
    
    # Universal Time
    TU = TL-C1-C2
    
    # Medium solar time
    
    TSM = TU-longitude/15
    Beta = (numday-1)*(360/365)
    ET = 229.2*(0.000075+0.001868*np.cos(Beta*np.pi/180)-0.032077*np.sin(Beta*np.pi/180)-0.014615*np.cos(2*Beta*np.pi/180)-0.04089*np.sin(2*Beta*np.pi/180));
    
    # Real Solar Time
    
    TSV = TSM+(ET/60)
    
    # Hour Angle
    
    omega = (TSV-12)*15
    
    # Solar Height
    
    sinh=np.cos(dec*np.pi/180)*np.cos(omega*np.pi/180)*np.cos(latitude*np.pi/180)+np.sin(dec*np.pi/180)*np.sin(latitude*np.pi/180)
    
    h=np.arcsin(sinh)*180/np.pi
    h[h < 0] = 0
    
    # Azimut Solar
    
    sina = np.cos(dec*np.pi/180)*np.sin(omega*np.pi/180)/np.cos(h*np.pi/180)
    a = np.arcsin(sina)*180/np.pi
    
    return omega, dec, h, a


def angle_incidence(dec, latitude, inclination, orientation, omega):
    cos_theta = np.sin(dec*np.pi/180)*np.sin(latitude*np.pi/180)*np.cos(inclination*np.pi/180)- \
        np.sin(dec*np.pi/180)*np.cos(latitude*np.pi/180)*np.sin(inclination*np.pi/180)* \
        np.cos(orientation*np.pi/180) + np.cos(dec*np.pi/180)*np.cos(latitude*np.pi/180)* \
        np.cos(inclination*np.pi/180)*np.cos(omega*np.pi/180) + np.cos(dec*np.pi/180)* \
        np.sin(latitude*np.pi/180)*np.sin(inclination*np.pi/180)*np.cos(orientation*np.pi/180)* \
        np.cos(omega*np.pi/180) + np.cos(dec*np.pi/180)*np.sin(inclination*np.pi/180)* \
        np.sin(orientation*np.pi/180)*np.sin(omega*np.pi/180);
    
    theta = np.arccos(cos_theta)*180/np.pi
    
    cos_thetaZ = np.cos(latitude*np.pi/180)*np.cos(dec*np.pi/180)*np.cos(omega*np.pi/180) + \
    np.sin(latitude*np.pi/180)*np.sin(dec*np.pi/180);
    
    thetaZ = np.arccos(cos_thetaZ)*180/np.pi
    
    return theta, thetaZ

### Feature Engineering Function

In [67]:
def feature_engineering(data, latitude, longitude, inclination, orientation, GMT_UTC, I0, DST):
    
    data_length = data.size
    
    seconds = np.zeros(data_length)
    
    # ------------ Deleting bad measurements ---------------
    data['DNI'] = pd.to_numeric(data['DNI'], downcast='float')
    data['Clearsky DHI'] = pd.to_numeric(data['Clearsky DHI'], downcast='float')
    data['DHI'] = pd.to_numeric(data['DHI'], downcast='float')
    data['GHI'] = pd.to_numeric(data['GHI'], downcast='float')
    data = data[ (data['GHI'] > 10) & (data['DHI'] > 10)]
    
    
    # ------------ First we get a date_serial number for the dates of the model --------------
    
    data['date'] = data.iloc[:, 0:5].apply(lambda row: '/'.join(row.values.astype(str)), axis=1)
    
    data['date']= pd.to_datetime(data.date, format = "%Y/%m/%d/%H/%M" )
    
    data['date_serial'] = data.date.apply(lambda day: excel_date(day))
    
    data.iloc[:,4] = pd.to_numeric(data.iloc[:,4] , downcast='float')
    data.iloc[:,1] = pd.to_numeric(data.iloc[:,1] , downcast='float')
    data.iloc[:,2] = pd.to_numeric(data.iloc[:,2] , downcast='float')
    data.iloc[:,3] = pd.to_numeric(data.iloc[:,3] , downcast='float')
    #---------- Calculating Legal Time -----------------
    
    HR = data.iloc[:, 3]
    MN = data.iloc[:, 4]
    MNhr = MN.div(60)
    data['TL'] = HR + MNhr
    
    #--------- Calculating Day Number ------------------
    
    mm = data.iloc[:,1];
    dd = data.iloc[:,2];
    numday = np.zeros(len(mm))
    for jj in range(0,len(mm)):
        if mm.iloc[jj] < 3:
            numday[jj] = dd.iloc[jj]+31*(mm.iloc[jj] - 1)
        else:
            numday[jj] = (dd.iloc[jj]+31*(mm.iloc[jj]-1)) - math.floor(0.4*mm.iloc[jj]+2.3)
    
    #---------- Calculating the solar coordinates, angle of incidience and zenith ------------
    # Omega: Solar Time, dec: Declination, h: solar height, a: Azimuth.
    data['omega'], data['dec'], data['h'], data['a'] = coord_solar(numday, data['TL'], longitude, latitude, DST)
    
    data['theta'], data['thetaZ'] = angle_incidence(data['dec'], latitude, inclination, orientation, data['omega'])
    
    #--------- Calculating the horizontal radiation outside the atmosphere ------------
    
    data['I'] = I0*(1.0+0.33*np.cos(360*numday*np.pi/(365.0*180.0)))*np.cos(data['thetaZ']*np.pi/180.0);
    
    #------------- Eliminating values of horizontal radiation outside atmosphere
    
    data = data[data['I'] > 0]
    
    #-------- Calculating Clarity index and eliminate data --------------------
    
    data['Mt'] = data['GHI'] / data['I']
    
    data = data[(data['Mt'] > 0) & (data['Mt'] < 1)]
    
    #------ Calculating Fraction of difuse radiation -------------------
    
    data['fd'] = data['DHI'] / abs(data['GHI'])
    
    
    return data



## Run the next code in order to generate the desired dataset

In [68]:
# ------------- Initial Constants ---------------------
I0 = 1367  #Solar constant W/m^2
DST=0

folder_name = 'Data_set_with_feature_engineering'
cities = ['BarrancoMinas', 'Bete', 'Bogota', 'Caruru', 'Chajal', 'Sipi', 'Taraira', 'PuertoMerizalde']

for city in cities:
    index_code_csv = find_csv_filenames(r'../{}'.format(city, current_year))[0][:-8]
    current_year = start_year
    appended_data = [] 
    while current_year <= end_year:

        txt = r'../{}/{}{}.csv'.format(city, index_code_csv, current_year)
        df = pd.read_csv(txt)
        if start_year == current_year:
            latitude = float(df.iloc[0,5])
            longitude = abs(float(df.iloc[0,6]))
            inclination = 0
            orientation = -90
            elevation = float(df.iloc[0,8])
            GMT_UTC = float(df.iloc[0,9])
        appended_data.append(df.loc[2::, :])
        header = df.loc[1, :]
        current_year+=1
    appended_data = pd.concat(appended_data)
    appended_data.columns = header
    appended_data = appended_data.iloc[:,0:22]
    
    data_with_features = feature_engineering(appended_data, latitude, longitude, inclination, orientation, GMT_UTC, I0, DST)
    training_data = data_with_features[['a', 'GHI', 'h', 'Mt', 'omega', 'dec', 'thetaZ', 'TL', 'DHI']]
    dataset_name = "{folder_name}/{train_or_test}/{train_or_test}_{city}_{start}_{end}.csv".format(folder_name=folder_name,
                                                                                               train_or_test=train_or_test, city=city, start=start_year, end=end_year)
    training_data = training_data.to_csv(dataset_name, index=False)

<ipython-input-67-4f504afdb85e>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'] = data.iloc[:, 0:5].apply(lambda row: '/'.join(row.values.astype(str)), axis=1)
<ipython-input-67-4f504afdb85e>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date']= pd.to_datetime(data.date, format = "%Y/%m/%d/%H/%M" )
<ipython-input-67-4f504afdb85e>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 